In [3]:
setwd(paste0(Sys.getenv('R_SOURCES'), '/house_prices'))
getwd()

[1] "/mnt/hdd/develop/kaggle/R_sources/house_prices"

In [9]:
source('main.R')
house_prices$helpers$import_libs()

combined_dataset <-
    house_prices$helpers$load_data() %>%
    (house_prices$outliers$remove_outliers) %>%
    (house_prices$missing$categ$replace_with_most_common) %>%
    (house_prices$missing$categ$fix_valid) %>%
    (house_prices$missing$numeric$replace_with_zero) %>%
    mutate(
        price_log = log(SalePrice)
    ) %>%
    select(-SalePrice, -Id)

training_dataset <- 
    combined_dataset %>% 
    filter(dataSource == 'train')


test_that("should be no NA values except SalePrice column", {
    expect_equal(sum(is.na(combined_dataset %>% select(-price_log))), 0)
})

test_that("should be no NA values in training_dataset", {
    expect_equal(sum(is.na(training_dataset)), 0)
})

In [239]:
combined_dataset %>%
select(house_prices$helpers$get_character_colnames(combined_dataset), price_log) %>%
filter(!is.na(price_log)) %>%
gather(var, value, -price_log) %>%
group_by(var, value) %>%
mutate(
    cdf = cume_dist(price_log)
) %>%
arrange(var, value, cdf)


price_log,var,value,cdf
10.86857,Alley,Grvl,0.02
10.97678,Alley,Grvl,0.04
11.07442,Alley,Grvl,0.06
11.11245,Alley,Grvl,0.08
11.19821,Alley,Grvl,0.10
11.37366,Alley,Grvl,0.12
11.40199,Alley,Grvl,0.14
11.40756,Alley,Grvl,0.16
11.47730,Alley,Grvl,0.18
11.49272,Alley,Grvl,0.20


In [234]:
source('main.R')

start_time <- Sys.time()
df1 <- 
    house_prices$trans$categ$calc_rating_for_selected(
        df = combined_dataset, 
        categ_vars_for_fix = house_prices$helpers$get_character_colnames(combined_dataset),
        target_var = price_log
    )
end_time <- Sys.time()
end_time - start_time


start_time <- Sys.time()
df2 <- 
    house_prices$trans$categ$calc_rating_for_selected2(
        df = combined_dataset, 
        categ_vars_for_fix = house_prices$helpers$get_character_colnames(combined_dataset),
        target_var = price_log
    )
end_time <- Sys.time()
end_time - start_time

test_that("should be equal ratings", {
    expect_equal(df1, df2)
})

Time difference of 0.2407243 secs

Time difference of 0.2554991 secs

In [235]:
house_prices$helpers$get_character_colnames(combined_dataset)

start_time <- Sys.time()
rating_for_selected <- 
    house_prices$trans$categ$calc_rating_for_selected(
        df = combined_dataset, 
        categ_vars_for_fix = house_prices$helpers$get_character_colnames(combined_dataset),
        target_var = price_log
    )
end_time <- Sys.time()
end_time - start_time

start_time <- Sys.time()
ratings_for_all <- 
    house_prices$trans$categ$calc_rating_for_all(combined_dataset, price_log)
end_time <- Sys.time()
end_time - start_time

test_that("should be equal ratings", {
    expect_equal(ratings_for_all, rating_for_selected)
})

ratings_for_all %>% head

[1] "Alley"         "BldgType"      "BsmtCond"      "BsmtExposure" 
 [5] "BsmtFinType1"  "BsmtFinType2"  "BsmtQual"      "CentralAir"   
 [9] "Condition1"    "Condition2"    "Electrical"    "ExterCond"    
[13] "Exterior1st"   "Exterior2nd"   "ExterQual"     "Fence"        
[17] "FireplaceQu"   "Foundation"    "Functional"    "GarageCond"   
[21] "GarageFinish"  "GarageQual"    "GarageType"    "Heating"      
[25] "HeatingQC"     "HouseStyle"    "KitchenQual"   "LandContour"  
[29] "LandSlope"     "LotConfig"     "LotShape"      "MasVnrType"   
[33] "MiscFeature"   "MSSubClass"    "MSZoning"      "Neighborhood" 
[37] "PavedDrive"    "PoolQC"        "RoofMatl"      "RoofStyle"    
[41] "SaleCondition" "SaleType"      "Street"        "Utilities"

Time difference of 0.2587614 secs

Time difference of 0.2474027 secs

var,value,rating
Alley,_none_,2.531822
Alley,Grvl,1.500000
Alley,Pave,2.536585
BldgType,1Fam,2.564039
BldgType,2fmCon,1.580645
BldgType,Duplex,1.692308


In [215]:
source('main.R')

ratings <- 
    house_prices$trans$categ$calc_rating_for_all(combined_dataset, price_log)

start_time <- Sys.time()
df1 <- 
    house_prices$trans$categ$rating_transform_for_selected(
        data = combined_dataset,
        columns = house_prices$helpers$get_character_colnames(combined_dataset),
        ratings = ratings
    )
end_time <- Sys.time()
end_time - start_time

start_time <- Sys.time()
df2 <- 
    house_prices$trans$categ$rating_transform(
        combined_dataset,
        price_log
    )
end_time <- Sys.time()
end_time - start_time

test_that("should be equal", {
    expect_equal(df1, df2)
})

Time difference of 0.08159423 secs

Time difference of 0.3141093 secs

In [211]:
source('main.R')

ratings <- 
    house_prices$trans$categ$calc_rating_for_all(combined_dataset, price_log)

start_time <- Sys.time()
df1 <- 
    house_prices$trans$categ$rating_transform_for_selected(
        data = combined_dataset,
        columns = house_prices$helpers$get_character_colnames(combined_dataset),
        ratings = ratings
    )
end_time <- Sys.time()
end_time - start_time


start_time <- Sys.time()
df2 <-
    house_prices$trans$categ$rating_transform_for_selected2(
        data = combined_dataset,
        columns = house_prices$helpers$get_character_colnames(combined_dataset),
        ratings = ratings
    )
end_time <- Sys.time()
end_time - start_time


start_time <- Sys.time()
df3 <-
    house_prices$trans$categ$rating_transform_for_selected3(
        data = combined_dataset,
        columns = house_prices$helpers$get_character_colnames(combined_dataset),
        ratings = ratings
    )
end_time <- Sys.time()
end_time - start_time


test_that("should be equal", {
    expect_equal(df1, df2)
})

test_that("should be equal", {
    expect_equal(df1, df3)
})

Time difference of 0.06634045 secs

Time difference of 0.7593565 secs

Time difference of 0.6167548 secs

In [188]:
aaa <- data_frame(x = c('a', 'a', 'b', 'b'), 
                  y = 1:4)
aaa
aaa %>% spread(x, y)

x,y
a,1
a,2
b,3
b,4


ERROR: Error: Duplicate identifiers for rows (1, 2), (3, 4)


In [186]:
aaa %>% 
group_by(x) %>% 
mutate(i = row_number()) %>% 
spread(x, y)

i,a,b
1,1,3
2,2,4


In [183]:
?dcast